<a href="https://colab.research.google.com/github/nboumlaik/Code2.0/blob/master/darling_dataprep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install gcsfs==0.5

In [0]:
pip install config

  Created wheel for config: filename=config-0.4.2-cp36-none-any.whl size=15135 sha256=2d774d684dd5da7bc353853997fb29d0237c326ab72aa73f1f8ff89a8bb5c5a4
  Stored in directory: /root/.cache/pip/wheels/51/7d/db/0e38d2ec57843d00cc39f8df3686984ccec689694f7bc78a38
Successfully built config


In [0]:
import pandas as pd
import uuid
from google.colab import auth
import os
from pathlib import Path
import gcsfs
import numpy as np
import ntpath

In [0]:
project_id = "darling-254913"
bucket_name = 'env_dev' + str(uuid.uuid1())

In [0]:
auth.authenticate_user()

In [0]:
!gcloud config set project {'darling-254913'}

Updated property [core/project].


To take a quick anonymous survey, run:
  $ gcloud survey



In [0]:
# data_root = Path("gs://env_dev/")
data_root = "gs://env_dev/"

In [0]:
def concat_col_names(cols1, cols2) -> list:
    ziplists = zip(cols1, cols2)
    res = []
    for i, j in ziplists:
      if j == '' or j == i:
        res += [i]
      else:
        res += ["%s_%s" % (i, j)]
    return res


def get_date_from_path(file_path) -> str:
    fname = ntpath.basename(file_path)
    name = os.path.splitext(fname)[0]
    name = name.replace('irk ', '')
    name = name.replace('v2', '')
    return name


def index_cleaner(table, cols_res_names) -> pd.DataFrame:
    index = table.index.tolist()
    index[1] = 'name2'
    table.index = index
    table = table.reset_index()
    table = table.dropna(axis=0, subset=['index'])
    table = table.set_index('index')
    table.loc['name2'] = cols_res_names
    return table


def merge(df, increment) -> pd.DataFrame:
    out = df.append(increment, sort=True)
    out = out[~out.index.duplicated()]
    return out.fillna(increment)

In [0]:
# sim_cov
path_listing = "%s%s" % (data_root, "Listing sociétés")
xlsx_files = gcsfs.GCSFileSystem(project_id).ls(path=path_listing)

df_sim_cov = None
cols = ['Company Name', 'Geographic Locations', 'Exchanges [Primary Listing]',
        'Equity Features', 'No of Analysts', 'Equity Type', 'Company Type',
        'Excel Company ID', 'Industry Classifications', 'Average - 1 Months']
for name in xlsx_files:
    full_path = 'gs://%s' % name
    oxlsx = pd.ExcelFile(full_path)
    sheet_names = [i for i in oxlsx.sheet_names if 'LISTING' in i.upper()]
    for sheet_name in sheet_names:
        dfderiv = oxlsx.parse(sheet_name=sheet_name, header=7)
        dfderiv = dfderiv[cols]
        dfderiv.dropna(how='all', inplace=True)
        new_col_name = 'Average - 1 Months Daily Value Traded'
        dfderiv = dfderiv.rename(columns={'Average - 1 Months': new_col_name})
        if df_sim_cov is None:
            df_sim_cov = dfderiv.copy()
        else:
            df_sim_cov = merge(df_sim_cov, dfderiv)

df_sim_cov = df_sim_cov.set_index('Company Name')
df_sim_cov.to_csv("%sBASE/sim_cov.csv" % data_root)

In [0]:
df_sim_cov.to_csv("%sBASE/sim_cov.csv" % data_root)

In [0]:
# path_red_irk = data_root / 'data' / 'data_IRK REF.xlsx' fonction pas
path_red_irk = "%s%s" % (data_root, "data/IRK REF.xlsx")
df_irk_ref = pd.read_excel(path_red_irk, index_col=0, header=None)
index = df_irk_ref.index.tolist()
index[2] = 'col_to_keep'
df_irk_ref.index = index
cols_ref_integer = df_irk_ref.dropna(axis=1, subset=['col_to_keep']).columns
path_target = "gs://env_dev/fichiers irk v2/"
list_xls_names = gcsfs.GCSFileSystem(project_id).ls(path=path_target)
cols_res_names = df_irk_ref.dropna(axis=1,
                                   subset=['col_to_keep']).loc['col_to_keep'].tolist()

In [0]:

list_name2 = df_irk_ref.loc['col_to_keep'].fillna('').tolist()
dict_res = {}
n = len(list_xls_names)
m = 1
for filepath in list_xls_names:
    if ".xlsx" in filepath:
        print('Processing for %s' % filepath)
        print('%s/%s' % (m, n))
        m += 1
        full_path = 'gs://%s' % filepath
        df = pd.read_excel(full_path, index_col=0, header=None,
                           sheet_name="Skynet")
        df = df[cols_ref_integer]
        df = index_cleaner(df, cols_res_names)
        list_name1 = df.loc['Nom'].tolist()
        list_name2 = df.loc['name2'].fillna('').tolist()

        new_cols = concat_col_names(list_name1, list_name2)
        df.columns = new_cols
        df.drop(labels=['Nom', 'name2'], axis=0, inplace=True)
        df = df.set_index('ISIN')
        str_date = get_date_from_path(filepath)
        dict_res[str_date] = df



In [0]:
dict_res_final = {}
for date, df in dict_res.items():
    print('Processing %s' % date)
    for col in df.columns:
        df_col = df[[col]].T
        df_col.index = [date]
        if col not in dict_res_final.keys():
            dict_res_final[col] = df_col
        else:
            df_col_in_dict = dict_res_final[col]
            dict_res_final[col] = merge(df_col_in_dict, df_col)


In [0]:
for col, df in dict_res_final.items():
    df.index = pd.to_datetime(df.index)
    try:
        df = df.astype('float')
    except:
        pass
    col = col.replace('/', '_')
    save_path = "gs://env_dev/BASE/AFRS/%s.csv" % col
    df.to_csv(save_path, float_format='%.6f')